In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import pickle

# print(tf.__version__)
# print(tf.keras.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# This code is partly based on a tutorial which can be found here:
# https://realpython.com/python-keras-text-classification/#what-is-a-word-embedding

In [ ]:
#Packages to import (regular expressions and pandas)

import re 
import pandas as pd 

from pathlib import Path
import gzip
import json

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Mount google colab to google drive so it can access files
from google.colab import drive
drive.mount('/content/drive')

#The huge gzipped JSON files ('NL_jobs.15.jsonl.gz') are located in this folder
cwd = '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

#Print the content of the folder
!ls '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

# set current working directory, This folder should only contain the huge json files.
%cd '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Baseline Model (BOW Log. regression).ipynb'
 BERT_alt_deploy.ipynb
'BERT_Jads_Nikita- Clyde edit - hpartuning.ipynb'
 BOW_LSTM.ipynb
'Copy of Copy of BERT_alt _ hyperparamtuning.ipynb'
 CORPUSNL_jobs.15.jsonl.gz_JSON_splitted.tsv_1.tsv
 Cow
 COW_LSTM_hyperparamtuning.ipynb
 dev.gsheet
 dev.tsv
 df3_updated_clean.txt
 df3_updated.txt
 df4_updated_clean.txt
 df4_updated.txt
 Exp23_RNDSCV.csv
 Exp25_RNDSCV.csv
 Exp26_RNDSCV.csv
 Exp27_RNDSCV.csv
 Exp28_RNDSCV.csv
 Exp29_RNDSCV.csv
 Exp30_RNDSCV.csv
 Exp31_RNDSCV.csv
 Exp32_RNDSCV.csv
 Exp33_RNDSCV.csv
 Exp34_RNDSCV.csv
 Exp35_RNDSCV.csv
 Exp36_RNDSCV.csv
 fastText-0.1.0
 glove.6B
 glove.6B.zip
 glove.model
 model.h5
 model.json
 NL_Embedding
 NL_jobs.15.jsonl.gz_JSON_splitted.tsv
 temp
 tokenizer.pickle
 train.gsheet
 train.tsv
 v0.1.0.zip
 v0.1.0.zip.1
 v0.1.0.zip.2
 Word2Vec_LSTM_hyperparamtuning.ipynb
 Word_

In [ ]:
TRAIN_FILE = "train.tsv"
DEV_FILE = "dev.tsv"

CORPUS_FILE = "CORPUSNL_jobs.15.jsonl.gz_JSON_splitted.tsv_1.tsv"
print("TRAIN FILE: \t\t{}\nDEVELOPMENT FILE: \t{}".format(TRAIN_FILE, DEV_FILE))

TRAIN FILE: 		train.tsv
DEVELOPMENT FILE: 	dev.tsv


In [ ]:
# Load the data
corpus_full = pd.read_csv(CORPUS_FILE, sep = "\t")

# ts = arr.tostring()
# Tirm the data to only one we need
corpus_train = np.array(corpus_full["#1 String"])

corpus_train2 = corpus_train.astype('str') 

# Seperate our data for training and testing
# train_text = (corpus_full["#1 String"])



In [ ]:
corpus_train2[0:100]

array(['Functiegroep', 'Techniek', 'Functie', 'werkvoorbereider',
       'Branche', 'Energie/Gas/Water', 'Telecom', 'Dienstverband', 'Uren',
       '36 - 40 uur per week', 'Opleidingsniveau', 'MBO',
       'Carrièreniveau', 'Starter', 'Salaris',
       '€ 2000 - 2500 bruto per maand', 'Adres', 'Middelweg 2',
       '5253CA Nieuwkuijk', 'Contactgegevens', 'Bas de Bruin',
       'Tel.: 0615951702', 'bdebruin@thepeoplegroup.nl',
       'Vacaturecode: 63559834 Meld Misbruik',
       'Waan je als tester in de wereld van CSI', 'Testing Professionals',
       '2500-3500', 'HBO', '40 uur', '40 uur', 'HBO', '2500-3500',
       'Vandaag', '2x bekeken', 'Functieomschrijving',
       'Je krijgt de kans om te werken in Team Software Engineering, bestaande uit 12 software ontwikkelaars en een aantal software testers.',
       "De testing kennis zit nu nog voornamelijk bij een aantal externe testers die worden ingehuurd, maar het doel is om deze kennis ook intern meer te borgen, middels het verder ui

In [ ]:
# Load the data
test_full = pd.read_csv(TRAIN_FILE, sep = "\t")
dev_full = pd.read_csv(DEV_FILE, sep = "\t")

# Tirm the data to only one we need
test_trim = test_full[["Quality", "#1 String"]]
dev_trim = dev_full[["Quality", "#1 String"]]

# Seperate our data for training and testing
train_text = np.array(test_full["#1 String"])
train_target = np.array(test_full["Quality"])

test_text = np.array(dev_full["#1 String"])
test_target = np.array(dev_full["Quality"])

# Check our training and testing sets
print("Train-set size: ", len(train_text))
print("Test-set size:  ", len(test_text))

# Stack text for tokenization
data_text = np.hstack((train_text, test_text))
print("Shape of the stacked text: {}".format(data_text.shape))

Train-set size:  2240
Test-set size:   720
Shape of the stacked text: (2960,)


In [ ]:
# Get stop words (or most commoon ones basically) for the dutch language
stop_words = set(stopwords.words('dutch'))

# Get the tokinized pattern of all sentences
word_tokens = []
for sentence in data_text:
    word_tokens.append(nltk.word_tokenize(sentence))

# Initiate a new collector
filtered_sentence = []
for entry in word_tokens:
    filtered_sentence.append([w for w in entry if not w in stop_words and w.isalpha()])
filtered_sentence = np.array(filtered_sentence)

# Implement the top level tokinizer, we will go for 10K words
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(filtered_sentence)

In [ ]:
# tokenizer.word_index # If we need to check it

In [ ]:
# Save the tokinizer for re-usability with the new data
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(train_text)
x_test_tokens = tokenizer.texts_to_sequences(test_text)

vocab_size = len(tokenizer.word_index) + 1  

# Example of conversions
print("Conversion Examples:")
print(x_train_tokens[50])
print(train_text[50])
print("---------------------------------")
print(x_train_tokens[100])
print(train_text[100])
print("---------------------------------")

# Padding the data if needed
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
print("Average Number of Tokens per sentence: ", np.round(np.mean(num_tokens), 4))
print("Maxmimum Number of Tokens per sentence: ", np.max(num_tokens))
print("Minimum Number of Tokens per sentence: ", np.min(num_tokens))

# The max number of tokens we will allow is set to the average plus 2 standard deviations.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print("Maximum amount of tokens to hold: ", max_tokens)

# This covers almost 96% of the data
print("How much data does the padding include fully: {}".format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

# Initiate the condition for padding
pad = 'pre'

# Applying padding to the training data
x_train_pad = pad_sequences(x_train_tokens,
                            maxlen=max_tokens,
                            padding=pad,
                            truncating=pad)

# Applying padding to the development data

x_test_pad = pad_sequences(x_test_tokens, 
                           maxlen=max_tokens, 
                           padding=pad, 
                           truncating=pad)

# Verify that all of the padding is done correctly
print("Shape of the padded testing data: \t", x_train_pad.shape)
print("Shape of the padded development data: \t", x_test_pad.shape)
# Seems correct

Conversion Examples:
[233, 2386, 245, 2387]
Utrecht, Drenthe en Flevoland.
---------------------------------
[13, 152, 614, 298, 1211, 2416, 298, 2417, 462, 237, 298, 2418, 835, 115, 823, 2419, 710]
het, met behulp van ons calculatieprogramma BV6 van TSD uit Zwolle, uitwerken van prijsaanvragen tot een correcte werkomschrijvende offerte
---------------------------------
Average Number of Tokens per sentence:  12.7966
Maxmimum Number of Tokens per sentence:  58
Minimum Number of Tokens per sentence:  0
Maximum amount of tokens to hold:  29
How much data does the padding include fully: 0.9618243243243243
Shape of the padded testing data: 	 (2240, 29)
Shape of the padded development data: 	 (720, 29)


In [ ]:
print("Shape of the padded testing data: \t", corpus_train_pad.shape)

Shape of the padded testing data: 	 (496016, 18)


In [ ]:
# corpus_train2[0:50]


list_train_text = corpus_train2.tolist()
print(type(list_train_text))
print(list_train_text[0:10])

s = ['hello everyone', 'how are you', 'i am fine']
word_list = [c.split() for c in list_train_text]
print(word_list) 

print(word_list[0:10])

<class 'list'>
['Functiegroep', 'Techniek', 'Functie', 'werkvoorbereider', 'Branche', 'Energie/Gas/Water', 'Telecom', 'Dienstverband', 'Uren', '36 - 40 uur per week']


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe


corpus.fit(word_list, window=10)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')
glove.add_dictionary(corpus.dictionary)


Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [ ]:
print(glove.word_vectors[glove.dictionary['Maximaal']])

[ 0.06591277  0.00852954 -0.05075319 -0.26351811  0.03398965]


In [ ]:
def create_model(lstm1, lstm2, dense1):
    #Creating the network
    # embedding_size = 256
    # num_words = num_words # We chose this value earlier (10,000)
    # max_tokens = max_tokens # This values is also chosen from before (30)

    # For reproducability setting a seed
    seed = 42
    # np.random.seed(seed)

    model = Sequential()
    model.add(Embedding(input_dim=194178,
                        output_dim=5,
                        weights = [glove.word_vectors], 
                        trainable = True,
                        input_length=29,
                        name='layer_embedding'))
    model.add(Bidirectional(LSTM(units = lstm1, return_sequences=True, name = "BiLSTM_1")))
    model.add(Dropout(0.5, noise_shape=None, seed=seed))
    model.add(Bidirectional(LSTM(units = lstm2, return_sequences=False, name = "BiLSTM_2")))
    model.add(Dropout(0.5, noise_shape=None, seed=seed))
    model.add(Dense(dense1, activation='relu', name = "Intermediate_1"))
    Dropout(rate = 0.5, noise_shape=None, seed=seed)
    # model.add(Dense(16, activation='relu', name = "Intermediate_2"))
    model.add(Dense(1, activation='sigmoid', name = "Final_Output"))
    optimizer = Adam(lr = 1e-3)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    # model.summary()

    return model

In [ ]:
param_grid = dict(lstm1=[4,8,16, 32],
                  lstm2=[4, 8,16, 32],
                  dense1=[4, 8,16, 32]         
                  # vocab_size=[5000], 
                  # embedding_dim=[50, 30, 80],
                )

# param_grid = dict(lstm1=[16, 32],
#                   lstm2=[16, 32],
#                   dense1=[16, 32],          
#                   vocab_size=[5000], 
#                   embedding_dim=[50],
#                   maxlen=[29],
#                    random_state=1, refit=True,
#                    return_train_score=True
#                   )


# param_grid = dict(batch_size_param=[16, 32, 64, 128],
#                   lr_param=[1e-3, 5e-5, 3e-5, 2e-5],          
#                   vocab_size=[5000], 
#                   embedding_dim=[50],
#                   maxlen=[29],       
#                    )


# param_grid = dict(num_filters=[32, 64, 128],
#                   kernel_size=[3, 5, 7],
#                   vocab_size=[vocab_size],
#                   embedding_dim=[embedding_dim],
#                   maxlen=[maxlen])

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV


output_file = 'data/output.txt'

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import fbeta_score, make_scorer


epochs = 15
cv_folds = 4
n_iter = 20
batch_size = 128

model = KerasClassifier(build_fn=create_model,
                        epochs=epochs, batch_size= batch_size,
                        verbose=False
                       )

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                          cv= cv_folds, verbose=1, n_iter= n_iter,
                          # scoring= score,
                          return_train_score = True,
                          n_jobs = -1,
                          refit = True
                          )



from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='loss', patience=5)


grid_result = grid.fit(x_train_pad, train_target, callbacks=[early_stopping])

# Evaluate testing set
test_accuracy = grid.score(x_test_pad, test_target)

print(grid_result.best_score_)
print(grid_result.best_params_)
# print(test_accuracy)
print(grid_result)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 25.7min finished


0.7799107134342194
{'lstm2': 16, 'lstm1': 4, 'dense1': 32}
RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc5d59db6a0>,
                   iid='deprecated', n_iter=20, n_jobs=-1,
                   param_distributions={'dense1': [4, 8, 16, 32],
                                        'lstm1': [4, 8, 16, 32],
                                        'lstm2': [4, 8, 16, 32]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=1)


In [ ]:
df_end = pd.DataFrame(grid_result.cv_results_)
df_end
# df_end.to_csv('Exp36_RNDSCV.csv')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lstm2,param_lstm1,param_dense1,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,35.715215,0.280475,2.916239,0.291775,4,32,16,"{'lstm2': 4, 'lstm1': 32, 'dense1': 16}",0.748214,0.773214,0.751786,0.703571,0.744196,0.025329,16,0.905357,0.869048,0.845833,0.867857,0.872024,0.021350
1,30.617582,0.816938,2.944659,0.295722,8,16,4,"{'lstm2': 8, 'lstm1': 16, 'dense1': 4}",0.750000,0.805357,0.766071,0.701786,0.755804,0.037123,10,0.888095,0.867262,0.886310,0.888095,0.882440,0.008794
2,35.943490,0.469996,2.909779,0.291271,8,32,8,"{'lstm2': 8, 'lstm1': 32, 'dense1': 8}",0.732143,0.796429,0.748214,0.733929,0.752679,0.026016,11,0.882738,0.832143,0.832738,0.872024,0.854911,0.022788
3,28.740405,1.097446,2.958508,0.257216,8,8,16,"{'lstm2': 8, 'lstm1': 8, 'dense1': 16}",0.778571,0.655357,0.796429,0.689286,0.729911,0.059163,20,0.919048,0.782143,0.870238,0.841071,0.853125,0.049554
4,31.760548,0.354100,2.933289,0.344224,8,16,8,"{'lstm2': 8, 'lstm1': 16, 'dense1': 8}",0.755357,0.762500,0.773214,0.703571,0.748661,0.026797,15,0.910119,0.838095,0.838095,0.885714,0.868006,0.031130
5,35.209018,0.957863,3.014147,0.174357,32,16,4,"{'lstm2': 32, 'lstm1': 16, 'dense1': 4}",0.742857,0.803571,0.814286,0.728571,0.772321,0.037148,2,0.875000,0.907738,0.878571,0.919643,0.895238,0.018968
6,28.662930,0.571588,3.023258,0.262464,4,4,8,"{'lstm2': 4, 'lstm1': 4, 'dense1': 8}",0.732143,0.792857,0.703571,0.701786,0.732589,0.036822,19,0.831548,0.814881,0.719048,0.810714,0.794048,0.043997
7,28.467727,0.742300,2.804130,0.376697,4,8,32,"{'lstm2': 4, 'lstm1': 8, 'dense1': 32}",0.691071,0.775000,0.810714,0.732143,0.752232,0.044952,12,0.824405,0.875595,0.917857,0.888690,0.876637,0.033814
8,31.745901,0.539628,3.289021,0.447765,16,8,16,"{'lstm2': 16, 'lstm1': 8, 'dense1': 16}",0.741071,0.780357,0.705357,0.733929,0.740179,0.026771,18,0.841071,0.811905,0.852381,0.909524,0.853720,0.035441
9,32.825748,1.294722,3.136111,0.326052,4,16,8,"{'lstm2': 4, 'lstm1': 16, 'dense1': 8}",0.739286,0.816071,0.798214,0.717857,0.767857,0.040505,4,0.905357,0.879762,0.891071,0.879167,0.888839,0.010651


In [ ]:
print(grid_result.best_score_)
print(grid_result.best_params_)

# test_accuracy = grid.score(x_test_pad, test_target)
# print(test_accuracy)
print(grid_result)


# train_accuracy = grid.score(x_train_pad, train_target)
# print(train_accuracy)

0.7799107134342194
{'lstm2': 16, 'lstm1': 4, 'dense1': 32}
RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc5d59db6a0>,
                   iid='deprecated', n_iter=20, n_jobs=-1,
                   param_distributions={'dense1': [4, 8, 16, 32],
                                        'lstm1': [4, 8, 16, 32],
                                        'lstm2': [4, 8, 16, 32]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=1)


In [ ]:
threshold = 0.5
predicted = grid.predict(x_train_pad)

predicted = np.where(predicted > threshold, 1,0)


predicted  = np.reshape(predicted, len(predicted))


eval_predicted = grid.predict(x_test_pad)

eval_predicted = np.where(eval_predicted > threshold, 1,0)

eval_predicted  = np.reshape(eval_predicted, len(eval_predicted))

# train acc

no_predictions = len(predicted)
print(no_predictions)

right = np.sum(predicted == train_target)

training_accuracy = right/ no_predictions
print("Training Accuracy: ", training_accuracy)


# test acc

no_predictions_eval = len(eval_predicted)
print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)

2240
Training Accuracy:  0.9254464285714286
720
Test Accuracy:  0.7222222222222222


In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk


precision = sk.precision_score(train_target, predicted)
print("Training Precision: ", precision)
recall = sk.recall_score(train_target, predicted)
print("Training recall: ", recall)
roc_auc = sk.roc_auc_score(train_target, predicted)
print("Training roc_auc: ", roc_auc)
f1 = sk.f1_score(train_target, predicted)
print("Training f1: ", f1)


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

Training Precision:  0.9022265246853823
Training recall:  0.9338677354709419
Training roc_auc:  0.9262736422926368
Training f1:  0.9177744953225012
Eval Precision:  0.6048632218844985
Eval recall:  0.7397769516728625
Eval roc_auc:  0.7257643073220188
Eval f1:  0.6655518394648829
